In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sps

filePath = "./data/data_train.csv"
targetFile = "./data/alg_sample_submission.csv"
df = pd.read_csv(filePath)
target_df = pd.read_csv(targetFile)
target_df

,user_id,item_list
0,0,0 1 2 3 4 5 6 7 8 9
1,1,0 1 2 3 4 5 6 7 8 9
2,2,0 1 2 3 4 5 6 7 8 9
3,4,0 1 2 3 4 5 6 7 8 9
4,5,0 1 2 3 4 5 6 7 8 9
...,...,...
28101,30906,0 1 2 3 4 5 6 7 8 9
28102,30907,0 1 2 3 4 5 6 7 8 9
28103,30908,0 1 2 3 4 5 6 7 8 9
28104,30909,0 1 2 3 4 5 6 7 8 9


In [11]:
ratingList = list(df['data'])
userList = list(df['row'])
itemList = list(df['col'])
targetList = list(target_df['user_id'])
len(targetList)

28106

In [3]:
URM_all = sps.coo_matrix((ratingList, (userList, itemList)))
URM_all = URM_all.tocsr()
URM_all

<30911x18495 sparse matrix of type '<class 'numpy.float64'>'
	with 398636 stored elements in Compressed Sparse Row format>

In [4]:
#warm_items_mask = np.ediff1d(URM_all.tocsc().indptr) > 0
#warm_items = np.arange(URM_all.shape[1])[warm_items_mask]

#URM_all = URM_all[:, warm_items]
#URM_all

In [5]:
#warm_users_mask = np.ediff1d(URM_all.tocsr().indptr) > 0
#warm_users = np.arange(URM_all.shape[0])[warm_users_mask]

#URM_all = URM_all[warm_users, :]
#URM_all

In [6]:
from Algorithms.Notebooks_utils.data_splitter import train_test_holdout

URM_train, URM_test = train_test_holdout(URM_all, train_perc = 0.8)
URM_train

<30911x18495 sparse matrix of type '<class 'numpy.float64'>'
	with 319079 stored elements in Compressed Sparse Row format>

In [7]:
from CF.user_cf import UserBasedCollaborativeFiltering
from CF.item_cf import ItemBasedCollaborativeFiltering

ub_recommender = UserBasedCollaborativeFiltering(URM_train, topK=100, shrink=20)
ub_recommender.fit()

ub_recommender.recommend(0, at=10)

UserBasedCF created!
Similarity column 30911 ( 100 % ), 1774.36 column/sec, elapsed time 0.29 min


array([ 3611,  6289,  6956,  4410,  8635,  3569,  8668, 15101, 13297,
       16360])

In [8]:
from Algorithms.Notebooks_utils.evaluation_function import evaluate_algorithm
#array = []

#for user_id in range(0, URM_all.shape[0]):
#    array.append(ub_recommender.recommend(user_id, at=10))

x_tick = [5]
MAP_per_k = []

for topK in x_tick:
    
    recommender = UserBasedCollaborativeFiltering(URM_train, topK=topK, shrink=20)
    recommender.fit()
    
    result_dict = evaluate_algorithm(URM_test, recommender)
    MAP_per_k.append(result_dict["MAP"])

UserBasedCF created!
Similarity column 30911 ( 100 % ), 1734.73 column/sec, elapsed time 0.30 min
Evaluated user 0 of 30911
Evaluated user 10000 of 30911
Evaluated user 20000 of 30911
Evaluated user 30000 of 30911
Recommender performance is: Precision = 0.0091, Recall = 0.0171, MAP = 0.0114


In [9]:
x_tick = [600]
MAP_per_shrinkage = []

for shrinkage in x_tick:
    
    recommender = UserBasedCollaborativeFiltering(URM_train, topK=10, shrink=shrinkage)
    recommender.fit()
    
    result_dict = evaluate_algorithm(URM_test, recommender)
    MAP_per_k.append(result_dict["MAP"])

UserBasedCF created!
Similarity column 30911 ( 100 % ), 1491.77 column/sec, elapsed time 0.35 min
Evaluated user 0 of 30911
Evaluated user 10000 of 30911
Evaluated user 20000 of 30911
Evaluated user 30000 of 30911
Recommender performance is: Precision = 0.0313, Recall = 0.0517, MAP = 0.0374


In [12]:
ub_recommender = UserBasedCollaborativeFiltering(URM_train, topK=5, shrink=500)
ub_recommender.fit()

array = []

for user_id in targetList:
    array.append(ub_recommender.recommend(user_id, at=10))

len(array)

UserBasedCF created!
Similarity column 30911 ( 100 % ), 1515.70 column/sec, elapsed time 0.34 min


28106

In [18]:
counter = 0

file = open("./output/file_out.csv", "w")

file.write("user_id,item_list\n")

for user_id in array:
    file.write(f'{targetList[counter]},')
    for i in range(0,10):
        if i != 9:
            file.write(f'{user_id[i]} ')
        else:
            file.write(f'{user_id[i]}')
    file.write("\n")
    counter += 1